# Logistic ~ Week 5
<hr>

## * Kelebihan:
 1. Accuracy cukup besar 0.99966
 2. Algoritma Mudah Dipahami
## * Kekurangan:
 1. Lemot ~ 1 jam 20 menit runtime.
<p style="font-size : 10pt">Note : Program yang lebih cepet & lebih akurat sama Dimas</p>

## Import Libraries
Import dulu gan ~

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import re
from tqdm import tqdm

## Loading Dataset
Load datanya~

In [2]:
data = pd.read_csv('../input/logistics-shopee-code-league/delivery_orders_march.csv')

## Checking NAN Value
Menghitung banyaknya NAN per kolom.

In [3]:
data.isna().sum()

orderid                      0
pick                         0
1st_deliver_attempt          0
2nd_deliver_attempt    1819311
buyeraddress                 0
selleraddress                0
dtype: int64

## Membuat Dataframe Expected Hari Pengiriman
Membuat dataframe yang berisi kota asal, kota tujuan, dan ketentuan waktu pengiriman sesuai data yang di excel.

In [4]:
city = list(map(lambda x : x.lower(), ['Metro Manila', 'Luzon', 'Visayas', 'Mindanao'])) # List kota yang ada
start, des = [], []
for i in range(len(city)):
    for j in range(i, len(city)):
        start.append(city[i])
        des.append(city[j])
target  = [3, 5, 7, 7, 5, 7, 7, 7, 7, 7]
kota = pd.DataFrame({'Awal' : start, 'Tujuan' : des, 'Target' : target})
kota

,Awal,Tujuan,Target
0,metro manila,metro manila,3
1,metro manila,luzon,5
2,metro manila,visayas,7
3,metro manila,mindanao,7
4,luzon,luzon,5
5,luzon,visayas,7
6,luzon,mindanao,7
7,visayas,visayas,7
8,visayas,mindanao,7
9,mindanao,mindanao,7


## Mengubah Format Timestamp Menjadi Datetime
Mengubah format timestamp menjadi datetime sesuai dengan ketentuan (`Tahun-Bulan-Tanggal` GMT +8).

```
GMT +8 Datetime format = to_datetime(Timestamp + 8 * 3600)
```

In [5]:
data['2nd_deliver_attempt'] = data['2nd_deliver_attempt'].fillna(0)   # Mengganti baris yang kosong pada kolom '2nd_deliver_attempt'2nd_deliver_attempt'
data['pick'] = pd.to_datetime(data['pick'].apply(lambda x : datetime.fromtimestamp(x + 8 * 3600).strftime('%Y-%m-%d')))
data['1st_deliver_attempt'] = pd.to_datetime(data['1st_deliver_attempt'].apply(lambda x : datetime.fromtimestamp(x + 8 * 3600).strftime('%Y-%m-%d')))
data['2nd_deliver_attempt'] = pd.to_datetime(data['2nd_deliver_attempt'].apply(lambda x : datetime.fromtimestamp(x + 8 * 3600).strftime('%Y-%m-%d')))
data.head()

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress
0,2215676524,2020-03-02,2020-03-05,1970-01-01,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...
1,2219624609,2020-03-04,2020-03-06,2020-03-10,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ..."
2,2220979489,2020-03-04,2020-03-06,1970-01-01,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M..."
3,2221066352,2020-03-05,2020-03-07,1970-01-01,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number..."
4,2222478803,2020-03-04,2020-03-06,1970-01-01,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...


## Memprediksi Telat Tidaknya Pengiriman
Main Idea:
### 1. Menghitung Expected Day
* Mencari Kota Asal dan Kota Tujuan pada setiap pengiriman.
* Memeriksa waktu pengiriman berdasarkan table Expected yang sudah dibuat.

### 2. Menghitung Banyak Hari Kerja
* Menghitung banyak hari kerja pengiriman dengan mengabaikan hari libur menggunakan fungsi `busday_count` dari `numpy`.

### 3. Membandingkan Waktu Pengiriman dengan Expected Day
* Jika Expected > Waktu pengiriman maka tidak telat `0`.
* Sebaliknya jika Expected < Waktu pengiriman maka telat `1`.

In [6]:
def Checker(start, end):
    start, end = start.strftime('%Y-%m-%d'), end.strftime('%Y-%m-%d')
    libur = ['2020-03-08', '2020-03-25', '2020-03-30', '2020-03-31']
    count = np.busday_count(start, end, weekmask = [1,1,1,1,1,1,0], holidays = libur)
    return count

In [7]:
res = []                                                            # Menginilisialisasi List untuk Is_Late
city_re = re.compile('|'.join(city))                                # Compiling RE untuk daftar kota"
for i in tqdm(range(len(data['buyeraddress']))):                    # Looping
    tujuan = city_re.findall(data['buyeraddress'][i].lower())[0]    # Mencari Kota Tujuan
    awal = city_re.findall(data['selleraddress'][i].lower())[0]     # Mencari Kota Akhir
    try:                                                            # Melihat Expected dari kora awal dan tujuan# Melihat Expected dari kora awal dan tujuan
        expected = kota['Target'][(kota['Awal'] == awal) & (kota['Tujuan'] == tujuan)].values[0]  
    except:
        expected = kota['Target'][(kota['Awal'] == tujuan) & (kota['Tujuan'] == awal)].values[0]
    
    # Menghitung banyak hari pengiriman pertama dan kedua# Menghitung banyak hari pengiriman pertama dan kedua
    count, second = Checker(data['pick'][i], data['1st_deliver_attempt'][i]), False
    if data['1st_deliver_attempt'][i] < data['2nd_deliver_attempt'][i]:
        second = Checker(data['1st_deliver_attempt'][i], data['2nd_deliver_attempt'][i])

    # Membandingkan Expected dengan banyak hari
    if count <= expected:                                           
        if second:
            if second <= 3:
                res.append(0)
            else:
                res.append(1)
        else:
            res.append(0)
    else:
        res.append(1)

 98%|█████████▊| 3122499/3176313 [58:55<00:59, 901.52it/s]

## Mebuat Dataframe Sesuai Ketentuan & Export ke CSV

In [8]:
submission = pd.DataFrame({'orderid' : data['orderid'].tolist(), 'is_late' : res})
submission.head(10)

,orderid,is_late
0,2215676524,0
1,2219624609,0
2,2220979489,0
3,2221066352,0
4,2222478803,0
5,2222597288,0
6,2222738456,0
7,2224695304,1
8,2224704587,0
9,2225138267,0


In [9]:
submission.to_csv('submission_1.csv', index = False)